# Data Prep - Visitors
Geoff Pidcock | PacifImpact
## Scope
Model visitor data for the following countries...
- FJ, Cook Island, Soloman Islands - MUST HAVE
- Vanuatu, Samoa - NICE TO HAVE

Document Assumptions in markdown as we go... <br>

and Save to PostGres <br>
Using the following data model
<br>
![Using the following data model](../references/ais-hack-country_metrics_schema.png)

## Reference 
- [modelling/survey gsheet](https://docs.google.com/spreadsheets/d/1qKgOixdJtwYD0jB1ouN4-pUIqrEXEPJzAqWfYbIY06g/edit#gid=0&fvid=1204782497)

# Setup

In [1]:
# Setup
import os
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

DBUNAME = os.environ.get("DBUNAME")
DBPASSWORD = os.environ.get("DBPASSWORD")
DBHOST = os.environ.get("DBHOST")
DBPORT = os.environ.get("DBPORT")
DBNAME = os.environ.get("DBNAME")

import sqlalchemy
import psycopg2
import pandas as pd
import numpy as np

connection_str = 'postgresql+psycopg2://'+DBUNAME+':'+DBPASSWORD+'@'+DBHOST+':'+DBPORT+'/'+DBNAME

In [ ]:
# creating the table (only execute this once!)

# with engine.connect() as con:
#     con.execute("""
#     drop table if exists public.country_metrics;
#     create table public.country_metrics
#     (
#         metric_key text not null
#         ,frequency text not null
#         ,country text not null
#         ,category text not null
#         ,name text not null
#         ,date date not null
#         ,value float8 null
#         ,properties json null
#         ,primary key(metric_key)
#     );;
#     """)

# Fiji (FJ)
**Source:** [Stats Fiji, Visitor Arrival Stats (accessed September 2020)](https://www.statsfiji.gov.fj/index.php/statistics/tourism-and-migration-statistics/visitor-arrivals-statistics)

Notes on usage
- Downloaded the "Visitor Arrivals Tables" file and Referred to `Table 1`
- Using a June snapshot with numbers up to June 2020
- Monthly aggregation
- Numbers are not seasonally adjusted
- Further commentary can be found [here](https://www.statsfiji.gov.fj/index.php/latest-releases/tourism-and-migration/visitor-arrivals/1087-provisional-visitor-arrivals-july-2020)

In [3]:
# evaluate dataset 1 - visitor arrival table
# source - https://www.statsfiji.gov.fj/index.php/component/advlisting/?view=download&format=raw&fileId=2148
# this was quite manual
fj_data = pd.read_excel('../data/raw/Tourism/Fiji-Visitor-Arrivals-Tables.xlsx'
                          ,sheet_name='T1'
                          ,usecols='B:F,H:J,L,N'
                          ,skiprows=15
                          ,nrows=32
                          ,header=None
                          ,names=['year','month','arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit','passenger-movement'])
display(fj_data.head(10),fj_data.tail(10))

,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement
0,2018.0,January,19324.0,62648.0,81972.0,15040.0,72627.0,87667.0,14484.0,184123.0
1,NaN,February,9650.0,48798.0,58448.0,12016.0,45491.0,57507.0,9385.0,125340.0
2,NaN,March,9676.0,60058.0,69734.0,12445.0,50698.0,63143.0,8304.0,141181.0
3,NaN,April,13739.0,63535.0,77274.0,13666.0,62345.0,76011.0,7699.0,160984.0
4,NaN,May,12975.0,67290.0,80265.0,12155.0,63679.0,75834.0,8210.0,164309.0
5,NaN,June,12681.0,81653.0,94334.0,13107.0,70851.0,83958.0,7813.0,186105.0
6,NaN,July,11502.0,95061.0,106563.0,14026.0,92072.0,106098.0,9648.0,222309.0
7,NaN,August,10418.0,88693.0,99111.0,14133.0,88552.0,102685.0,8007.0,209803.0
8,NaN,September,10198.0,81437.0,91635.0,13729.0,76513.0,90242.0,9700.0,191577.0
9,NaN,October,12667.0,79077.0,91744.0,12341.0,81194.0,93535.0,9200.0,194479.0


,year,month,arrivals-resident,arrivals-visitor,arrivals-total,departures-resident,departures-visitors,departures-total,total-in-transit,passenger-movement
22,NaN,October,10396.0,77467.0,87863.0,12610.0,80129.0,92739.0,9594.0,190196.0
23,NaN,November,9688.0,69123.0,78811.0,17822.0,68917.0,86739.0,7350.0,172900.0
24,NaN,December,18465.0,73740.0,92205.0,22026.0,61528.0,83554.0,13034.0,188793.0
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2020.0,January,18238.0,65386.0,83624.0,15434.0,76195.0,91629.0,12571.0,187824.0
27,NaN,February,10448.0,46343.0,56791.0,11748.0,43748.0,55496.0,8366.0,120653.0
28,NaN,March,8166.0,27972.0,36138.0,7608.0,34782.0,42390.0,4064.0,82592.0
29,NaN,April,510.0,678.0,1188.0,532.0,1488.0,2020.0,28.0,3236.0
30,NaN,May,410.0,709.0,1119.0,259.0,940.0,1199.0,0.0,2318.0
31,NaN,June,628.0,413.0,1041.0,650.0,645.0,1295.0,0.0,2336.0


##### proposed prep steps
- remove nulls
- forward fill year
- cast year and month as date
- melt
- append with metadata
- check and cast types

In [4]:
# remove nulls (excepting year)
fj_data_prep = fj_data.copy()
fj_data_prep.dropna(axis=0,subset=['month'],inplace=True)
display(fj_data_prep.shape,fj_data.shape)
# output makes sense as there are two breaks for years

(30, 10)

(32, 10)

In [5]:
# forward fill year
fj_data_prep.loc[:,'year'] = fj_data_prep.loc[:,'year'].ffill()
# fj_data_prep #dev check

In [6]:
# cast year and month as date
fj_data_prep['year'] = fj_data_prep['year'].astype('int')
fj_data_prep['year_month'] = fj_data_prep['year'].astype('str') +' '+ fj_data_prep['month']
fj_data_prep['date'] = pd.to_datetime(fj_data_prep['year_month'],format='%Y %B')
# fj_data_prep.head() #devcheck

In [7]:
# melt
fj_data_prep2 = fj_data_prep.copy()
fj_data_prep2 = fj_data_prep2.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit','passenger-movement']
                  ,var_name='name'
                  ,value_name='value'
                  )
# fj_data_prep2 #dev check

In [8]:
# append with metadata
properties = "{'currency': ''}"
fj_data_prep2['properties'] = properties
fj_data_prep2['frequency'] = 'monthly'
fj_data_prep2['country'] = 'fj'
fj_data_prep2['category'] = 'tourism'
fj_data_prep2['source'] = 'stats-fiji'
fj_data_prep2['metric_key'] = fj_data_prep2['country']+'-'+fj_data_prep2['name']+'-'+fj_data_prep2['date'].astype('str')
fj_data_prep2 = fj_data_prep2[['metric_key','frequency','country','category','source','name','date','value','properties']]
fj_data_prep2

,metric_key,frequency,country,category,source,name,date,value,properties
0,fj-arrivals-resident-2018-01-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-01-01,19324.0,{'currency': ''}
1,fj-arrivals-resident-2018-02-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-02-01,9650.0,{'currency': ''}
2,fj-arrivals-resident-2018-03-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-03-01,9676.0,{'currency': ''}
3,fj-arrivals-resident-2018-04-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-04-01,13739.0,{'currency': ''}
4,fj-arrivals-resident-2018-05-01,monthly,fj,tourism,stats-fiji,arrivals-resident,2018-05-01,12975.0,{'currency': ''}
...,...,...,...,...,...,...,...,...,...
235,fj-passenger-movement-2020-02-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-02-01,120653.0,{'currency': ''}
236,fj-passenger-movement-2020-03-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-03-01,82592.0,{'currency': ''}
237,fj-passenger-movement-2020-04-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-04-01,3236.0,{'currency': ''}
238,fj-passenger-movement-2020-05-01,monthly,fj,tourism,stats-fiji,passenger-movement,2020-05-01,2318.0,{'currency': ''}


In [9]:
# check and cast types
fj_data_prep2.info()
# all good :)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  240 non-null    object        
 1   frequency   240 non-null    object        
 2   country     240 non-null    object        
 3   category    240 non-null    object        
 4   source      240 non-null    object        
 5   name        240 non-null    object        
 6   date        240 non-null    datetime64[ns]
 7   value       240 non-null    float64       
 8   properties  240 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 17.0+ KB


# Cook Islands CK
**Source:** [Cook Islands Ministry of Finance & Economic Management, Tourism and Migration Statistics](http://www.mfem.gov.ck/statistics/social-statistics/tourism-and-migration)

Notes on Usage
- Downloaded the `Mig_Statistics_Tables_
- Referred to `Table 1` (Total Arrivals and Departures) from row 519 down (Jan 2018 to Mar 2020)
   - It is missing 3 months of data relative to FJ
   - It is missing `total-in-transit` and `passenger-movement` 
      - `total-in-transit` can be manually calculated. 
- Monthly frequency
- Non seasonally adjusted

In [10]:
# evaluate dataset 1 - visitor arrival table
# source - https://www.statsfiji.gov.fj/index.php/component/advlisting/?view=download&format=raw&fileId=2148
# this was quite manual
ck_data = pd.read_excel('../data/raw/Tourism/Cook Islands - Mig_Statistics_Tables_202003.xlsx'
                          ,sheet_name='Table 1'
                          ,usecols='A,D:E,H:I,L:M'
                          ,skiprows=519
                          ,nrows=29
                          ,header=None
                          ,names=['month','arrivals-total','departures-total','arrivals-visitor'
                                  ,'departures-visitors','arrivals-resident','departures-resident'])
display(ck_data.head(),ck_data.tail())

,month,arrivals-total,departures-total,arrivals-visitor,departures-visitors,arrivals-resident,departures-resident
0,Jan(p),11093.0,14823.0,9527.0,13405.0,1566.0,1418.0
1,Feb(p),9556.0,9809.0,8459.0,8938.0,1097.0,871.0
2,Mar(p),13155.0,12419.0,11963.0,11374.0,1192.0,1045.0
3,Apr(p),14691.0,14686.0,13231.0,13634.0,1460.0,1052.0
4,May(p),16108.0,15048.0,15171.0,14087.0,937.0,961.0


,month,arrivals-total,departures-total,arrivals-visitor,departures-visitors,arrivals-resident,departures-resident
24,Dec(p),15796.0,13039.0,14567.0,10692.0,1229.0,2347.0
25,2020,NaN,NaN,NaN,NaN,NaN,NaN
26,Jan(p),11854.0,15760.0,9986.0,14252.0,1868.0,1508.0
27,Feb(p),10131.0,10252.0,8928.0,9341.0,1203.0,911.0
28,Mar(p),6598.0,8033.0,5814.0,7357.0,784.0,676.0


##### proposed prep steps
- remove nulls
- manually assign years
- calculate `total-in-transit`
- cast year and month as date
- melt
- append with metadata

In [11]:
# remove nulls
ck_data_prep = ck_data.copy()
ck_data_prep.dropna(axis=0,subset=['arrivals-total'],inplace=True)
ck_data_prep.reset_index(inplace=True,drop=True)
display(ck_data_prep.shape,ck_data.shape)
# all good

(27, 7)

(29, 7)

In [12]:
# manually assign years
ck_data_prep['year'] = ''
ck_data_prep.loc[0:12,'year']='2018'
ck_data_prep.loc[12:24,'year']='2019'
ck_data_prep.loc[24:,'year']='2020'

In [13]:
# calculate `total-in-transit`
ck_data_prep['total-in-transit'] = ck_data_prep['arrivals-total'] + ck_data_prep['departures-total']

In [14]:
# cast year and month as date
ck_data_prep['year_month'] = ck_data_prep.apply(lambda x: x['year']+' '+x['month'][:3], axis=1)
ck_data_prep['date'] = pd.to_datetime(ck_data_prep['year_month'],format='%Y %b')
# ck_data_prep.head()

In [15]:
# melt
ck_data_prep2 = ck_data_prep.copy()
ck_data_prep2 = ck_data_prep2.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total',
                                 'departures-resident','departures-visitors','departures-total',
                                 'total-in-transit']
                  ,var_name='name'
                  ,value_name='value'
                  )
# ck_data_prep2 #dev check

In [16]:
# append with metadata
properties = "{'currency': ''}"
ck_data_prep2['properties'] = properties
ck_data_prep2['frequency'] = 'monthly'
ck_data_prep2['country'] = 'ck'
ck_data_prep2['category'] = 'tourism'
ck_data_prep2['source'] = 'mfem-ck'
ck_data_prep2['metric_key'] = ck_data_prep2['country']+'-'+ck_data_prep2['name']+'-'+ck_data_prep2['date'].astype('str')
ck_data_prep2 = ck_data_prep2[['metric_key','frequency','country','category','source','name','date','value','properties']]
ck_data_prep2.head()

,metric_key,frequency,country,category,source,name,date,value,properties
0,ck-arrivals-resident-2018-01-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-01-01,1566.0,{'currency': ''}
1,ck-arrivals-resident-2018-02-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-02-01,1097.0,{'currency': ''}
2,ck-arrivals-resident-2018-03-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-03-01,1192.0,{'currency': ''}
3,ck-arrivals-resident-2018-04-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-04-01,1460.0,{'currency': ''}
4,ck-arrivals-resident-2018-05-01,monthly,ck,tourism,mfem-ck,arrivals-resident,2018-05-01,937.0,{'currency': ''}


In [17]:
# check and cast types
ck_data_prep2.info()
# all good :)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  189 non-null    object        
 1   frequency   189 non-null    object        
 2   country     189 non-null    object        
 3   category    189 non-null    object        
 4   source      189 non-null    object        
 5   name        189 non-null    object        
 6   date        189 non-null    datetime64[ns]
 7   value       189 non-null    float64       
 8   properties  189 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 13.4+ KB


# Solomon Islands (SB)
**Source:** [Solomon Islands National Statistics Office, Visitor Arrivals Statistics First Quarter 2020 (Accessed Sept 2020](https://www.statistics.gov.sb/images/SolomonFiles/Social-and-Demography-Statistics/Visitors/2020/Visitor_Arrivals_Statistics_1st-Quarter_2020.pdf)

Handling Notes:
- Made the news - "Tourist Arrivals Fall 48.6%" - [link](https://www.solomontimes.com/news/tourist-arrivals-fall-486-due-to-covid19/9898)
- Data is quarterly
- Data has the following limitations
  - Only goes up to March 2020
  - Only contains arrivals
  - Splits resident and intended resident- sum to resident for purpose of reporting. 
- PDF
  - copied to clipboard and cleaned in a text file, named `sb_visitors.txt`

In [18]:
# sb_data = pd.read_clipboard(sep=' ',header=None) # read clipboard AFTER manually correcting spacing in a text file >_< 
sb_data = pd.read_csv('../data/raw/Tourism/sb_visitors.txt',sep=' ',header=None) # refactored to read from text directly
display(sb_data.head(),sb_data.tail())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2016,1,"2,651","1,654","4,305",5,5,10,"4,315",301,101,402,"3,603","1,434","5,037","6,560","3,194","9,754"
1,2016,2,"3,547","1,945","5,492",11,7,18,"5,510",291,66,357,"3,917","1,762","5,679","7,766","3,780","11,546"
2,2016,3,"2,977","1,845","4,822",6,4,10,"4,832",230,83,313,"3,892","1,773","5,665","7,105","3,705","10,810"
3,2016,4,"4,685","2,933","7,618",19,4,23,"7,641",142,42,184,"4,449","2,364","6,813","9,295","5,343","14,638"
4,2017,1,"3,183","1,711","4,894",3,3,6,"4,900",199,47,246,"3,485","1,396","4,881","6,870","3,157","10,027"


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
12,2019,1,"3,713","2,019","5,732",1,0,1,"5,733",74,23,97,"4,351","1,668","6,019","8,139","3,710","11,849"
13,2019,2,"4,129","2,445","6,574",2,1,3,"6,577",74,33,107,"4,726","2,337","7,063","8,931","4,816","13,747"
14,2019,3,"4,185","2,209","6,394",1,1,2,"6,396",51,31,82,"5,026","2,690","7,716","9,263","4,931","14,194"
15,2019,4,"4,909","3,142","8,051",1,0,1,"8,052",46,26,72,"5,255","2,854","8,109","10,211","6,022","16,233"
16,2020,1,"2,612","1,579","4,191",0,0,0,"4,191",48,21,69,"2,870","1,210","4,080","5,530","2,810","8,340"


##### proposed prep steps
Where to start...
- name and/or drop columns
- cast year and quarter as date
- apply assumption in calculating full arrivals-resident
- melt
- append with metadata

In [19]:
# name and drop columns
sb_data_prep = sb_data.loc[:,[0,1,8,11,14,17]]

sb_data_prep.columns = ['year','quarter','arrivals-resident-1','arrivals-resident-2'
                       ,'arrivals-visitor','arrivals-total']

# sb_data_prep

In [20]:
# cast year and quarter as date
def quarter_mapping(q):
    mapping = {
        1: 'Jan'
        ,2: 'Apr'
        ,3: 'Jul'
        ,4:'Oct'
    }
    return mapping.get(q)
sb_data_prep['month'] = sb_data_prep.quarter.apply(lambda x: quarter_mapping(x))
sb_data_prep['year_month'] = sb_data_prep['year'].astype('str') +' '+ sb_data_prep['month']
sb_data_prep['date'] = pd.to_datetime(sb_data_prep['year_month'],format='%Y %b')
# sb_data_prep

In [21]:
# apply assumption in calculating full arrivals-resident
sb_data_prep_2 = sb_data_prep.copy()
for c in ['arrivals-resident-1','arrivals-resident-2','arrivals-visitor','arrivals-total']:
    sb_data_prep_2[c] = pd.to_numeric(sb_data_prep_2[c].astype(str).str.replace(',',''), errors='coerce')

sb_data_prep_2['arrivals-resident'] = sb_data_prep_2['arrivals-resident-1'] + sb_data_prep_2['arrivals-resident-2']
# sb_data_prep_2

In [22]:
# melt
sb_data_prep3 = sb_data_prep_2.copy()
sb_data_prep3 = sb_data_prep3.melt(id_vars = ['date']
                  ,value_vars=['arrivals-resident','arrivals-visitor','arrivals-total']
                  ,var_name='name'
                  ,value_name='value'
                  )
# sb_data_prep3.head() #dev check

In [23]:
# append with metadata
properties = "{'currency': ''}"
sb_data_prep3['properties'] = properties
sb_data_prep3['frequency'] = 'quarterly'
sb_data_prep3['country'] = 'sb'
sb_data_prep3['category'] = 'tourism'
sb_data_prep3['source'] = 'stats-sb'
sb_data_prep3['metric_key'] = sb_data_prep3['country']+'-'+sb_data_prep3['frequency']+'-'+sb_data_prep3['name']+'-'+sb_data_prep3['date'].astype('str')
sb_data_prep3 = sb_data_prep3[['metric_key','frequency','country','category','source','name','date','value','properties']]
sb_data_prep3.head()

,metric_key,frequency,country,category,source,name,date,value,properties
0,sb-quarterly-arrivals-resident-2016-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-01-01,4717,{'currency': ''}
1,sb-quarterly-arrivals-resident-2016-04-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-04-01,5867,{'currency': ''}
2,sb-quarterly-arrivals-resident-2016-07-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-07-01,5145,{'currency': ''}
3,sb-quarterly-arrivals-resident-2016-10-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2016-10-01,7825,{'currency': ''}
4,sb-quarterly-arrivals-resident-2017-01-01,quarterly,sb,tourism,stats-sb,arrivals-resident,2017-01-01,5146,{'currency': ''}


In [24]:
# one last check of info
sb_data_prep3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  51 non-null     object        
 1   frequency   51 non-null     object        
 2   country     51 non-null     object        
 3   category    51 non-null     object        
 4   source      51 non-null     object        
 5   name        51 non-null     object        
 6   date        51 non-null     datetime64[ns]
 7   value       51 non-null     int64         
 8   properties  51 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.7+ KB


# Writing to DB
1. copy the current status of the prod table to staging
2. make our changes to staging/dev
3. test our changes to staging/dev
4. copy our changes back to the prod table

In [25]:
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn = engine.connect()
except:
    print('Database connection error - check creds')

In [26]:
# confirm tables are present as expected
engine.table_names() 

['test', 'country_metrics', 'test2', 'stg_country_metrics']

In [27]:
data_write = pd.concat([sb_data_prep3,ck_data_prep2,fj_data_prep2])
data_write.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  480 non-null    object        
 1   frequency   480 non-null    object        
 2   country     480 non-null    object        
 3   category    480 non-null    object        
 4   source      480 non-null    object        
 5   name        480 non-null    object        
 6   date        480 non-null    datetime64[ns]
 7   value       480 non-null    float64       
 8   properties  480 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 37.5+ KB


In [28]:
# Write step 1: copy current status of prod to staging
with engine.connect() as con:
    con.execute("""
    drop table if exists public.stg_country_metrics
    ;
    create table public.stg_country_metrics as 
    select * from public.country_metrics
    ;;
    """)

In [30]:
# Write step 2: make changes to staging

## 2.1 determine which rows of your data are actually new, using the key
data_sql = pd.read_sql(
    """
    select * 
    from public.country_metrics 
    where category = 'tourism'
    """,con=engine)
data_write_new = pd.concat(
    [data_write, data_sql]).drop_duplicates(
    subset=['metric_key'], keep=False)
data_write_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   metric_key  480 non-null    object        
 1   frequency   480 non-null    object        
 2   country     480 non-null    object        
 3   category    480 non-null    object        
 4   source      480 non-null    object        
 5   name        480 non-null    object        
 6   date        480 non-null    datetime64[ns]
 7   value       480 non-null    float64       
 8   properties  480 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 37.5+ KB


In [31]:
## 2.2 write changes
data_write_new.to_sql('stg_country_metrics'
                 ,con = engine
                 ,if_exists='append'
                 ,index=False
                 ,method='multi')

In [32]:
# 3 test our changes
pd.read_sql("""
select country, category, name, count(date) as count_measure
from public.country_metrics
group by 1,2,3 order by 1,2,3

""",con= engine)

,country,category,name,count_measure
0,ck,GDP,GDP_Accomodation Services,12
1,ck,GDP,GDP_Agriculture,12
2,ck,GDP,"GDP_Arts, Recreation, Other Services",12
3,ck,GDP,GDP_Construction,12
4,ck,GDP,GDP_Education,12
...,...,...,...,...
116,wsm,GDP,GDP_Ownership of Dwellings,10
117,wsm,GDP,GDP_Personal & Other Services,10
118,wsm,GDP,GDP_Public Administration,10
119,wsm,GDP,GDP_Total Value added at market prices,10


In [33]:
pd.read_sql("""
select distinct country, category
from public.country_metrics
""",con= engine)

,country,category
0,vn,trade
1,kir,CPI
2,sb,trade
3,ck,GDP
4,wsm,GDP
5,fj,trade
6,sb,CPI
7,ck,trade
8,PW,CPI
9,ws,trade


#### estimate change
- new category - tourism

In [34]:
pd.read_sql("""
select country, category, name, count(date) as count_measure
from public.stg_country_metrics
where category = 'tourism'
group by 1,2,3 order by 1,2,3

""",con= engine)

,country,category,name,count_measure
0,ck,tourism,arrivals-resident,27
1,ck,tourism,arrivals-total,27
2,ck,tourism,arrivals-visitor,27
3,ck,tourism,departures-resident,27
4,ck,tourism,departures-total,27
5,ck,tourism,departures-visitors,27
6,ck,tourism,total-in-transit,27
7,fj,tourism,arrivals-resident,30
8,fj,tourism,arrivals-total,30
9,fj,tourism,arrivals-visitor,30


In [36]:
pd.read_sql("""
select count(*) as row_count
from public.stg_country_metrics
where category <> 'tourism'
union all
select count(*) as row_count
from public.country_metrics
where category <> 'tourism'
""",con=engine)

,row_count
0,2673
1,2673


In [37]:
# 4. copy changes back to prod
with engine.connect() as con:
    con.execute("""
    drop table if exists public.country_metrics
    ;
    create table public.country_metrics as 
    select * from public.stg_country_metrics
    ;;
    """)